# Compare Gridding Rate

In [ ]:
import fsspec
import radarx as rx
import numpy as np
import xradar as xd
import cmweather  # noqa
import cartopy.crs as ccrs

In [ ]:
def filter_radar(ds):
    ds = ds.where((ds.DBZH >= 0) & (ds.DBZH <= 70))
    return ds

## Load data

In [ ]:
file = "s3://noaa-nexrad-level2/2018/06/12/KSGF/KSGF20180612_083109_V06"
local_file = fsspec.open_local(
    f"simplecache::s3://{file}",
    s3={"anon": True},
    filecache={"cache_storage": "."},
)

In [ ]:
dtree = xd.io.open_nexradlevel2_datatree(local_file)

In [ ]:
dtree = rx.utils.combine_nexrad_sweeps(dtree)

In [ ]:
dtree = dtree.xradar.map_over_sweeps(filter_radar)

In [ ]:
dtree = dtree.xradar.georeference()
dtree.groups

In [ ]:
dtree["sweep_0"]["DBZH"].plot(
    x="x",
    y="y",
    xlim=(-300e3, 200e3),
    ylim=(-200e3, 300e3),
    cmap="ChaseSpectral",
    levels=range(-10, 70),
)

## Grid Setup

In [ ]:
x_lims = (-300e3, 250e3)
y_lims = (-250e3, 300e3)
z_lims = (0, 20e3)
h_res = 1000
v_res = 250

## Py-ART

In [ ]:
%%time
radar = dtree.pyart.to_radar(scan_type="ppi")

In [ ]:
%%time
# Grid using 11 vertical levels, and 101 horizontal grid cells at a resolution on 1 km
grid = pyart.map.grid_from_radars(
    (radar,),
    grid_shape=(81, 551, 551),
    grid_limits=(
        z_lims,
        y_lims,
        x_lims,
    ),
    fields=["DBZH"],
)

xg = grid.to_xarray()

display(xg)

## Radarx

In [ ]:
%%time
ds_rx = dtree.radarx.to_grid(
    data_vars=["DBZH"],
    pseudo_cappi=False,
    x_lim=x_lims,
    y_lim=y_lims,
    z_lim=z_lims,
    x_step=h_res,
    y_step=h_res,
    z_step=v_res,
    x_smth=1,
    y_smth=1,
    z_smth=4,
)

display(ds_rx)

## Wradlib

In [ ]:
%%time
# Grid setup
sitecoords = (
    dtree["sweep_0"].longitude.values,
    dtree["sweep_0"].latitude.values,
    dtree["sweep_0"].altitude.values,
)

proj = osr.SpatialReference()
proj.ImportFromEPSG(4326)
maxrange = 275e3
maxalt = 20e3
horiz_res = 1000
vert_res = 250
minalt = dtree.altitude.values
minelev = 0.2
maxelev = 21.0

# Create target 3D grid
trgxyz, trgshape = wrl.vpr.make_3d_grid(
    sitecoords, proj, maxrange, maxalt, horiz_res, vert_res
)

print(trgshape)

In [ ]:
%%time
raw_dt = dtree.xradar.map_over_sweeps(rx.utils.get_geocoords)

proj_crs = xd.georeference.get_crs(raw_dt["sweep_0"].ds)

proj = osr.SpatialReference()

proj.ImportFromEPSG(4326)

swp_list = []
for swp in raw_dt.match("sweep_*"):
    ds = raw_dt[swp].to_dataset()
    xyz = (
        xr.concat(
            [
                ds.coords["x"].reset_coords(drop=True),
                ds.coords["y"].reset_coords(drop=True),
                ds.coords["z"].reset_coords(drop=True),
            ],
            "xyz",
        )
        .stack(npoints=("azimuth", "range"))
        .transpose(..., "xyz")
    )
    swp_list.append(xyz)
xyz = xr.concat(swp_list, "npoints")

data_list = []
for key in list(raw_dt.children):
    if "sweep" in key:
        ds = raw_dt[key].ds
        data = ds.DBZH.stack(npoints=("azimuth", "range"))
        data_list.append(data)
data = xr.concat(data_list, "npoints")

# interpolate to Cartesian 3-D volume grid
gridder = wrl.vpr.CAPPI(
    xyz.values,
    trgxyz,
    # gridshape=trgshape,
    maxrange=maxrange,
    minelev=minelev,
    maxelev=maxelev,
)

vol = np.ma.masked_invalid(gridder(data.values).reshape(trgshape))

# diagnostic plot
trgx = trgxyz[:, 0].reshape(trgshape)[0, 0, :]
trgy = trgxyz[:, 1].reshape(trgshape)[0, :, 0]
trgz = trgxyz[:, 2].reshape(trgshape)[:, 0, 0]

ds_wrl = xr.DataArray(
    data=vol, coords={"z": trgz, "y": trgy, "x": trgx}, dims=("z", "y", "x"), name="REF"
).to_dataset()
ds_wrl["time"] = ds_rx["time"]
ds_wrl.attrs = ds_rx.attrs
ds_wrl["latitude"] = ds_rx["latitude"]
ds_wrl["longitude"] = ds_rx["longitude"]
ds_wrl["lon"] = xr.DataArray(ds_rx.lon.values, dims=["x"])
ds_wrl["lat"] = xr.DataArray(ds_rx.lat.values, dims=["y"])
ds_wrl = ds_wrl.set_coords(["lon", "lat"])

display(ds_wrl)

## Plot

Lets plot all three grids

## Radarx

In [ ]:
ds_rx.radarx.plot_max_cappi(
    data_var="DBZH",
    vmin=-10,
    vmax=70,
    range_rings=True,
    add_map=True,
    projection=ccrs.PlateCarree(),
)

## Py-ART

In [ ]:
xg.squeeze().radarx.plot_max_cappi(
    data_var="DBZH",
    vmin=-10,
    vmax=70,
    range_rings=True,
    add_map=True,
    projection=ccrs.PlateCarree(),
)

## Wradlib

In [ ]:
ds_wrl.radarx.plot_max_cappi(
    data_var="REF",
    vmin=-10,
    vmax=70,
    range_rings=True,
    add_map=True,
)